# SZ22, E. coli credentialed data



In [1]:
!pip install mass2chem 

In [2]:
import numpy as np
import json
import pandas as pd

from scipy import stats 

from mass2chem.epdsConstructor import epdsConstructor

from matplotlib import pyplot as plt
# import seaborn 

%matplotlib inline

In [3]:
# from JMS
def read_table_to_peaks(infile, 
                        has_header=True, mz_col=1, rtime_col=2, feature_id=None,
                        full_extract=False,
                        delimiter='\t'):
    '''
    Read a text feature table, and 
    return list of peaks, e.g. [ {
        'id_number': 555,
        'mz': 133.0970, 
        'apex': 654, 
        'height': 14388.0, 
        }, ... ]
    
    full_extract: to keep all fields in output as strings, only if has_header.
    feature_id: if None, create id for each peak/feature.
    '''
    def _make_id(ii, mz, rt):
        return 'F' + str(ii) + '_' + str(round(mz, 6)) + '@' + str(round(rt, 2))

    list_peaks = []
    w = open(infile).readlines()
    if has_header:
        header = w[0].rstrip().split(delimiter)
        w = w[1:]
    ii = 0
    for line in w:
        a = line.split(delimiter)   # not rstrip, so trailing EOL will be carried forward
        mz, rt = float(a[mz_col]), float(a[rtime_col])
        if feature_id != None:
            fid = a[feature_id].strip()
        else:
            ii += 1
            fid = _make_id(ii, mz, rt)
        peak = {'id_number': fid, 'mz': mz, 'rtime': rt, 'apex': rt}
        if has_header and full_extract:
            peak2 = dict(zip(header, a))
            peak2.update(peak)
            peak = peak2

        list_peaks.append( peak )

    return list_peaks

In [4]:
!ls ecoli

asari_cred.tsv				 ratios_xcms_RPpos_creds.xlsx
asari_SZ22_RPpos_full_Feature_table.tsv  run_xcms_SZ22.R
ratios_asari_RPpos_creds.csv		 SZ22_RPpos_XCMS_featureTable.csv
ratios_asari_RPpos_creds.xlsx		 xcms_cred.tsv
ratios_xcms_RPpos_creds.csv


In [5]:
asari_table = pd.read_csv("ecoli/asari_SZ22_RPpos_full_Feature_table.tsv", index_col=0, header=0, sep='\t')
asari_table

,mz,rtime,rtime_left_base,rtime_right_base,parent_masstrack_id,peak_area,cSelectivity,goodness_fitting,snr,detection_counts,13C_Ecoli_20220321_004_20220322101150,12C_Ecoli_20220321_004,12C_Ecoli_20220321_004_20220322095030,12C_Ecoli_20220321_004_20220322130235,13C_Ecoli_20220321_004,13C_Ecoli_20220321_004_20220322132355
id_number,,,,,,,,,,,,,,,,
F1,95.9735,125.06,83.20,166.12,276,8230447,0.98,0.61,10,6,1159692,2079328,984349,1314348,1707804,1299125
F2,61.9278,40.94,38.27,41.92,0,312476,0.97,0.44,11,4,57714,42543,0,0,43369,60144
F3,115.9802,11.42,0.54,30.54,548,20375423,1.00,0.42,14,6,3326312,3362695,3228128,4076328,3201740,3942341
F4,115.9860,5.83,0.54,25.49,549,11153489,1.00,0.79,26,6,1642861,2035714,1739145,1978764,1977817,2040421
F5,115.9860,27.96,25.49,30.10,549,1123274,1.00,0.69,15,6,185255,182603,219669,248607,199092,185630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F3598,285.1692,30.54,30.10,32.33,4299,424430,1.00,0.54,27,1,0,0,0,0,0,411079
F3599,134.1046,104.21,98.34,109.06,3739,476867,1.00,0.70,12,3,0,0,125475,176741,0,154015
F3600,155.0104,35.46,34.03,36.16,3754,274424,0.40,0.37,10,0,0,0,0,0,0,0


In [6]:
list_peaks = read_table_to_peaks("ecoli/asari_SZ22_RPpos_full_Feature_table.tsv", 
                        has_header=True, mz_col=1, rtime_col=2, feature_id=0,
                        full_extract=False,
                        delimiter='\t')
print(len(list_peaks), list_peaks[5])

3602 {'id_number': 'F6', 'mz': 133.0649, 'rtime': 42.17, 'apex': 42.17}


In [7]:
ECCON = epdsConstructor(list_peaks, mode='pos')
dict_empCpds = ECCON.peaks_to_epdDict(
        seed_search_patterns = ECCON.seed_search_patterns, 
        ext_search_patterns = ECCON.ext_search_patterns,
        mz_tolerance_ppm=5, 
        coelution_function='distance',
        check_isotope_ratio = False
) 



Annotating empirical compounds on 3602 features/peaks, ...
epdsConstructor - numbers of seeded epds and included peaks:  (409, 932)


In [8]:
dict_empCpds[55]

{'interim_id': 55,
 'neutral_formula_mass': None,
 'neutral_formula': None,
 'Database_referred': [],
 'identity': [],
 'MS1_pseudo_Spectra': [{'id_number': 'F309',
   'mz': 118.9776,
   'rtime': 61.52,
   'apex': 61.52,
   'ion_relation': 'anchor'},
  {'id_number': 'F382',
   'mz': 119.9807,
   'rtime': 60.06,
   'apex': 60.06,
   'ion_relation': '13C/12C'},
  {'id_number': 'F261',
   'mz': 136.9883,
   'rtime': 54.41,
   'apex': 54.41,
   'ion_relation': 'anchor,+H2O'},
  {'id_number': 'F262',
   'mz': 136.9883,
   'rtime': 64.01,
   'apex': 64.01,
   'ion_relation': 'anchor,+H2O'}],
 'MS2_Spectra': []}

In [9]:
use_features, use_epds = [], []
for v in dict_empCpds.values():
    ion_relations = [x.get('ion_relation', '') for x in v['MS1_pseudo_Spectra']]
    if "13C/12C" in ion_relations:
        use_features.append(v['MS1_pseudo_Spectra'][0]['id_number'])
        use_epds.append(v)

print(len(use_features), use_features[:5])

253 ['F15', 'F27', 'F44', 'F45', 'F52']


In [10]:
creds = []
for C in use_epds:
    d = {}
    for peak in C['MS1_pseudo_Spectra']:
        d[peak['ion_relation']] = peak['id_number']
        
    try:
        creds.append((d['anchor'], d['13C/12C']))
    except KeyError:
        print(d)
    
print(len(creds), creds[:3])

{'13C/12C': 'F96', 'anchor,+H2O': 'F1678'}
{'13C/12C': 'F1322'}
{'13C/12C': 'F2372', 'Na/H': 'F1325', '13C/12C,39K/H': 'F2124'}
{'Na/H': 'F2999', '13C/12C': 'F3046', 'Na/H,Na/H, double charged': 'F3415'}
{'H': 'F3261', '13C/12C': 'F3264', '13C/12C,Acetonitrile': 'F3404'}
248 [('F15', 'F136'), ('F60', 'F27'), ('F44', 'F79')]


In [11]:
asari_table.columns

Index(['mz', 'rtime', 'rtime_left_base', 'rtime_right_base',
       'parent_masstrack_id', 'peak_area', 'cSelectivity', 'goodness_fitting',
       'snr', 'detection_counts', '13C_Ecoli_20220321_004_20220322101150',
       '12C_Ecoli_20220321_004', '12C_Ecoli_20220321_004_20220322095030',
       '12C_Ecoli_20220321_004_20220322130235', '13C_Ecoli_20220321_004',
       '13C_Ecoli_20220321_004_20220322132355'],
      dtype='object')

In [12]:
C12 = ['12C_Ecoli_20220321_004',
       '12C_Ecoli_20220321_004_20220322095030',
       '12C_Ecoli_20220321_004_20220322130235',]
C13 = ['13C_Ecoli_20220321_004',
       '13C_Ecoli_20220321_004_20220322101150',
       '13C_Ecoli_20220321_004_20220322132355',]

In [13]:
pair1 = [x[0] for x in creds]
pair2 = [x[1] for x in creds]

data1 = asari_table[C12 + C13].loc[pair1, :]
data2 = asari_table[C12 + C13].loc[pair2, :]

# data1 have 12C peaks in all 6 samples; data2 13C

## select credentialed features

In Mahieu et al 2014, the samples were mixed at specific ratios.

Our 12C and 13C samples were analyzed separately.

to call a credentialed feature, we require 
- (Feature_13C_unlabeled: Feature_12C_unlabeled) <= 1 (= because both can be missing)
- (Feature_13C_labeled: Feature_12C_labeled)/(Feature_13C_unlabeled: Feature_12C_unlabeled) > 2




In [14]:
data1.to_csv("data1.tsv", sep="\t")
data2.to_csv("data2.tsv", sep="\t")

In [18]:
# Manually edited two tables into one: ecoli/asari_cred.tsv

Ndata = pd.read_csv('ecoli/asari_cred.tsv', index_col=0, header=0, sep="\t")

count = Ndata[Ndata>1].count(axis='columns')
Ndata['detection_counts'] = count

Ndata = Ndata[Ndata['detection_counts'] > 11]

Ndata

,12C_Ecoli_20220321_004,12C_Ecoli_20220321_004_20220322095030,12C_Ecoli_20220321_004_20220322130235,13C_Ecoli_20220321_004,13C_Ecoli_20220321_004_20220322101150,13C_Ecoli_20220321_004_20220322132355,12C_Ecoli_a,12C_Ecoli_b,12C_Ecoli_c,13C_Ecoli_a,13C_Ecoli_b,13C_Ecoli_c,detection_counts
id_number,,,,,,,,,,,,,
F44,151998387,159288756,137455404,161488773,152909184,137151280,125199,301853,217885,72210,152668,167346,12
F45,53221677,84354887,77646261,20481890,21381814,20919628,1848384,183515,2088978,120561,184393,190855,12
F72,1719283462,1488674573,1010324382,1811272395,1363333313,978051854,2211725,1625797,1682689,2134514,1595910,1693058,12
F76,387642,372886,362583,397482,324959,390103,1918801,2178378,1835371,2132995,2321761,2018808,12
F77,176706,1422483,807810,194714,166123,114366,799491,1613574,2222528,760287,460771,248471,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
F2886,5568288,1205098,1433741,4758047,1193449,1560575,609337,148114,146678,498886,140562,189704,12
F2923,282180,285904,255256,288224,294914,304948,441462,464202,518981,439253,396505,529441,12
F3041,923579,972898,1088808,906066,953567,1125138,1465321,1273356,1610692,1323812,1264131,1672011,12


In [19]:
# Ndata = Ndata.fillna(0)

In [20]:
Ndata.columns[:3]

Index(['12C_Ecoli_20220321_004', '12C_Ecoli_20220321_004_20220322095030',
       '12C_Ecoli_20220321_004_20220322130235'],
      dtype='object')

In [21]:
# ratio calculation

FF = 100   # Add a stablizing number, because ratio calculation is unreliable when peak intensities are low.

GA = Ndata.columns[9:12]  #13 peaks in labeled samples
GB = Ndata.columns[3:6 ]
ratios13 = []

GA12 = Ndata.columns[6:9] #13 peaks in unlabeled samples
GB12 = Ndata.columns[:3]   

ratios12 = [] 
RRS = []

for index, row in Ndata.iterrows():
    a = sum(row[GA12] + FF)/ sum(row[GB12] + FF)
    b = sum(row[GA] + FF)/ sum(row[GB] + FF)
    ratios12.append( a )
    ratios13.append( b )
    RRS.append(b/a)
    
    
Ndata['ratios_unlabeled'] = ratios12
Ndata['ratios_labeled'] = ratios13
Ndata['RRS'] = RRS

Ndata.to_csv("ratios_asari_RPpos_creds.csv")




In [23]:
# Got 143 credentialed features 
# Manually retrieved from spreadsheet.

cDict = dict(creds)

In [24]:
asari_features = """F1101
F273
F2713
F1597
F1447
F1446
F609
F186
F1490
F2409
F2357
F1210
F1465
F1445
F524
F1136
F1106
F974
F1923
F198
F788
F1737
F848
F2869
F2050
F1326
F1518
F1779
F497
F2886""".split()

# then get the 13C partners

all_asari_cred = []
for x in asari_features:
    all_asari_cred += [x, cDict[x]]
    

In [25]:
len(all_asari_cred), all_asari_cred[:8]

(60, ['F1101', 'F27', 'F273', 'F346', 'F2713', 'F2718', 'F1597', 'F1676'])

## We got 60 credentialed features in asari result

Now onto XCMS result

In [26]:
xcms_table = pd.read_csv("ecoli/SZ22_RPpos_XCMS_featureTable.csv", index_col=0, header=0,)
xcms_table = xcms_table.fillna(0)
xcms_table

,mzmed,mzmin,mzmax,rtmed,rtmin,rtmax,npeaks,X12C_Ecoli_20220321_004_20220322095030.mzML,X12C_Ecoli_20220321_004_20220322130235.mzML,X12C_Ecoli_20220321_004.mzML,X13C_Ecoli_20220321_004_20220322101150.mzML,X13C_Ecoli_20220321_004_20220322132355.mzML,X13C_Ecoli_20220321_004.mzML
FT0001,61.927787,61.927787,61.927787,17.251722,17.251722,17.251722,1,7123.804449,0.000000,0.000000,0.000000,0.000000,0.000000
FT0002,62.051181,62.051181,62.051181,24.452236,24.452236,24.452236,1,0.000000,0.000000,0.000000,17185.875523,0.000000,0.000000
FT0003,62.098197,62.098159,62.098216,29.940554,29.775288,30.043172,3,28506.908086,0.000000,25227.421208,16494.047689,0.000000,0.000000
FT0004,65.010928,65.010928,65.010928,18.212740,18.212740,18.212740,1,8136.009134,0.000000,0.000000,0.000000,0.000000,0.000000
FT0005,66.033868,66.033842,66.033894,131.768318,129.254333,134.282303,2,69965.485464,0.000000,0.000000,0.000000,0.000000,66028.965148
...,...,...,...,...,...,...,...,...,...,...,...,...,...
FT4750,702.211924,702.211924,702.211924,212.114914,212.114914,212.114914,1,0.000000,4067.282716,0.000000,0.000000,0.000000,0.000000
FT4751,758.569022,758.569022,758.569022,196.320267,196.320267,196.320267,1,0.000000,19375.677032,0.000000,0.000000,0.000000,0.000000
FT4752,759.572589,759.572589,759.572589,196.320267,196.320267,196.320267,1,0.000000,8437.890509,0.000000,0.000000,0.000000,0.000000
FT4753,774.562487,774.562487,774.562487,195.012543,195.012543,195.012543,1,0.000000,0.000000,0.000000,4841.619461,0.000000,0.000000


In [27]:
list_peaks2 = read_table_to_peaks("ecoli/SZ22_RPpos_XCMS_featureTable.csv", 
                        has_header=True, mz_col=1, rtime_col=4, feature_id=0,
                        full_extract=False,
                        delimiter=',')
print(len(list_peaks2), list_peaks2[5])

4754 {'id_number': '"FT0006"', 'mz': 66.0338189315886, 'rtime': 149.593322753906, 'apex': 149.593322753906}


In [28]:
ECCON = epdsConstructor(list_peaks2, mode='pos')
dict_empCpds2 = ECCON.peaks_to_epdDict(
        seed_search_patterns = ECCON.seed_search_patterns, 
        ext_search_patterns = ECCON.ext_search_patterns,
        mz_tolerance_ppm=5, 
        coelution_function='distance',
        check_isotope_ratio = False
) 



Annotating empirical compounds on 4754 features/peaks, ...
epdsConstructor - numbers of seeded epds and included peaks:  (775, 1699)


In [29]:
dict_empCpds2[55]

{'interim_id': 55,
 'neutral_formula_mass': None,
 'neutral_formula': None,
 'Database_referred': [],
 'identity': [],
 'MS1_pseudo_Spectra': [{'id_number': '"FT0498"',
   'mz': 102.009186260366,
   'rtime': 18.3397682520635,
   'apex': 18.3397682520635,
   'ion_relation': 'anchor'},
  {'id_number': '"FT0520"',
   'mz': 103.012589253303,
   'rtime': 18.953092717103,
   'apex': 18.953092717103,
   'ion_relation': '13C/12C'},
  {'id_number': '"FT0700"',
   'mz': 114.003707942245,
   'rtime': 17.596471786499,
   'apex': 17.596471786499,
   'ion_relation': '13C/12C,Na/H, double charged'}],
 'MS2_Spectra': []}

In [30]:
use_features, use_epds = [], []
for v in dict_empCpds2.values():
    ion_relations = [x.get('ion_relation', '') for x in v['MS1_pseudo_Spectra']]
    if "13C/12C" in ion_relations:
        use_features.append(v['MS1_pseudo_Spectra'][0]['id_number'])
        use_epds.append(v)

print(len(use_features), use_features[:5])

477 ['"FT0026"', '"FT0040"', '"FT0047"', '"FT0072"', '"FT0081"']


In [31]:
xcreds = []
for C in use_epds:
    d = {}
    for peak in C['MS1_pseudo_Spectra']:
        d[peak['ion_relation']] = peak['id_number']
        
    try:
        xcreds.append((d['anchor'].replace('"', ''), d['13C/12C'].replace('"', '')))
    except KeyError:
        print(d)
    
print(len(xcreds), xcreds[:3])

477 [('FT0026', 'FT0036'), ('FT0040', 'FT0048'), ('FT0047', 'FT0057')]


In [32]:
xcms_table.columns

Index(['mzmed', 'mzmin', 'mzmax', 'rtmed', 'rtmin', 'rtmax', 'npeaks',
       'X12C_Ecoli_20220321_004_20220322095030.mzML',
       'X12C_Ecoli_20220321_004_20220322130235.mzML',
       'X12C_Ecoli_20220321_004.mzML',
       'X13C_Ecoli_20220321_004_20220322101150.mzML',
       'X13C_Ecoli_20220321_004_20220322132355.mzML',
       'X13C_Ecoli_20220321_004.mzML'],
      dtype='object')

In [33]:
C12 = ['12C_Ecoli_20220321_004',
       '12C_Ecoli_20220321_004_20220322095030',
       '12C_Ecoli_20220321_004_20220322130235',]
C13 = ['13C_Ecoli_20220321_004',
       '13C_Ecoli_20220321_004_20220322101150',
       '13C_Ecoli_20220321_004_20220322132355',]

In [34]:
C12 = ['X'+x+'.mzML' for x in C12]
C13 = ['X'+x+'.mzML' for x in C13]

pair1 = [x[0] for x in xcreds]
pair2 = [x[1] for x in xcreds]

data1 = xcms_table[C12 + C13].loc[pair1, :]
data2 = xcms_table[C12 + C13].loc[pair2, :]

data1.to_csv("data1.tsv", sep="\t")
data2.to_csv("data2.tsv", sep="\t")

In [35]:
# Manually edited two tables into one: ecoli/xcms_cred.tsv

Ndata = pd.read_csv('ecoli/xcms_cred.tsv', index_col=0, header=0, sep="\t")

count = Ndata[Ndata>1].count(axis='columns')
Ndata['detection_counts'] = count

Ndata = Ndata[Ndata['detection_counts'] > 11]

Ndata.shape

(23, 13)

In [36]:
# ratio calculation

FF = 100   # Add a stablizing number, because ratio calculation is unreliable when peak intensities are low.

GA = Ndata.columns[9:12]  #13 peaks in labeled samples
GB = Ndata.columns[3:6 ]
ratios13 = []

GA12 = Ndata.columns[6:9] #13 peaks in unlabeled samples
GB12 = Ndata.columns[:3]   

ratios12 = [] 
RRS = []

for index, row in Ndata.iterrows():
    a = sum(row[GA12] + FF)/ sum(row[GB12] + FF)
    b = sum(row[GA] + FF)/ sum(row[GB] + FF)
    ratios12.append( a )
    ratios13.append( b )
    RRS.append(b/a)
    
    
Ndata['ratios_unlabeled'] = ratios12
Ndata['ratios_labeled'] = ratios13
Ndata['RRS'] = RRS

Ndata.to_csv("ratios_xcms_RPpos_creds.csv")



In [37]:
# Got 310 features 
# Manually retrieved from spreadsheet.

xDict = dict(xcreds)

In [38]:
xcms_features = """FT1248
FT2460
FT4460
FT2604
FT4516
FT1710
FT2766
FT3548
FT2023
FT1044
FT1394""".split()

# then get the 13C partners

all_xcms_cred = []
for x in xcms_features:
    all_xcms_cred += [x, xDict[x]]
    

In [39]:
len(all_xcms_cred), all_xcms_cred[:8]

(22,
 ['FT1248',
  'FT1282',
  'FT2460',
  'FT2476',
  'FT4460',
  'FT4464',
  'FT2604',
  'FT2622'])

## We got 22 credentialed features in XCMS result

Now combine the data to create ground truth set of features.

First test how many overlap.

In [40]:
# match functions
# 0.000010 is 10 ppm
PPM_tolerance = 0.000005
# use a large number to include anything in RTime, small number to be specific
RTime_tolerance = 10       # seconds in retention time, usually a small number
                            # more lenient for diff instruments
                            # and possible diff void volume
             
# F1 = (m/z, rt)
def match2(F1, F2, PPM_tolerance=PPM_tolerance, RTime_tolerance=RTime_tolerance):
    if abs(F1[0]-F2[0])/F1[0] < PPM_tolerance and abs(F1[1] - F2[1]) < RTime_tolerance:
        return True
    else:
        return False
    
# test
match2((129.1541, 55), (129.1538, 14))

False

In [41]:
len(all_asari_cred), len(all_xcms_cred)

(60, 22)

In [42]:
len(set(all_asari_cred)), len(set(all_xcms_cred))

(59, 22)

In [43]:
asari_table = pd.read_csv("ecoli/asari_SZ22_RPpos_full_Feature_table.tsv", index_col=0, header=0, sep='\t')
xcms_table = pd.read_csv("ecoli/SZ22_RPpos_XCMS_featureTable.csv", index_col=0, header=0,)

asari_table = asari_table.loc[all_asari_cred]
xcms_table = xcms_table.loc[all_xcms_cred]    # ['"'+x+'"' for x in all_xcms_cred]

In [44]:
mz_a, rt_a = asari_table['mz'].to_list(), asari_table['rtime'].to_list()
mz_x, rt_x = xcms_table['mzmed'].to_list(), xcms_table['rtmed'].to_list()
N1_, N2_ = len(mz_a), len(mz_x)
print(N1_, N2_)

good = []
for ii in range(N1_):
    F1 = (mz_a[ii], rt_a[ii])
    for jj in range(N2_):
        F2 = (mz_x[jj], rt_x[jj])
        if match2(F1, F2):
            good.append((ii, jj))

60 22


In [45]:
len(good)

8

In [46]:
# not in overlap but in XCMS
set2 = set([x[1] for x in good])
print(len(set2))
xcms_only = [jj for jj in range(N2_) if jj not in set2]
print(len(xcms_only), xcms_only[:5])


8
14 [2, 3, 4, 5, 8]


In [47]:
# not in overlap but in asari
set1 = set([x[0] for x in good])
print(len(set1))
asari_only = [ii for ii in range(N1_) if ii not in set1]
print(len(asari_only), asari_only[:5])


8
52 [0, 1, 4, 5, 6]


**This means ground truth features are 82, 8 of which overlap**



In [48]:
len(mz_a), len(mz_x)

(60, 22)

In [49]:
true_mzs = mz_a + mz_x
true_rts = rt_a + rt_x

true_features = list(zip(true_mzs, true_rts))

len(true_features), true_features[:5]

(82,
 [(132.1214, 25.05),
  (133.1248, 24.82),
  (137.0457, 30.54),
  (138.0488, 30.54),
  (314.3422, 129.37)])

In [55]:
true_features.sort()
s = 'mz\trt\n'
for T in true_features:
    s += str(T[0]) + '\t' + str(T[1]) + '\n'
with open("Ecoli_ground_truth_features.txt", "w") as O:
    O.write(s)

In [56]:
asari_table = pd.read_csv("ecoli/asari_SZ22_RPpos_full_Feature_table.tsv", index_col=0, header=0, sep='\t')
xcms_table = pd.read_csv("ecoli/SZ22_RPpos_XCMS_featureTable.csv", index_col=0, header=0,)


In [57]:
mz_a, rt_a = asari_table['mz'].to_list(), asari_table['rtime'].to_list()
mz_x, rt_x = true_mzs, true_rts

N1_, N2_ = len(mz_a), len(mz_x)
print(N1_, N2_)

good = []
for ii in range(N1_):
    F1 = (mz_a[ii], rt_a[ii])
    for jj in range(N2_):
        F2 = (mz_x[jj], rt_x[jj])
        if match2(F1, F2):
            good.append((ii, jj))
            
print(len(good),
     len(set([x[0] for x in good])),
     )

found_true = set([x[1] for x in good])
print("~~~~  ", len(found_true), "   ~~~~  ")
for ii in range(N2_):
    if ii not in found_true:
        print( (mz_x[ii], rt_x[ii]) )

3602 82
91 78
~~~~   79    ~~~~  
(199.99920579749298, 1.04856933095572)
(418.220827711138, 124.892322255)
(291.851048148417, 21.5713682174683)


The above 3 missed peaks: two are too below min peak height;

one has bad shape, missing 1st half


In [58]:
mz_a, rt_a = xcms_table['mzmed'].to_list(), xcms_table['rtmed'].to_list()

mz_x, rt_x = true_mzs, true_rts

N1_, N2_ = len(mz_a), len(mz_x)
print(N1_, N2_)

good = []
for ii in range(N1_):
    F1 = (mz_a[ii], rt_a[ii])
    for jj in range(N2_):
        F2 = (mz_x[jj], rt_x[jj])
        if match2(F1, F2):
            good.append((ii, jj))
            
print(len(good),
     len(set([x[0] for x in good])),
     )

found_true = set([x[1] for x in good])
print("~~~~  ", len(found_true), "   ~~~~  ")
for ii in range(N2_):
    if ii not in found_true:
        print( (mz_x[ii], rt_x[ii]) )

4754 82
72 63
~~~~   68    ~~~~  
(192.9756, 15.79)
(142.9621, 68.65)
(99.9647, 27.73)
(152.9883, 16.04)
(192.9756, 15.79)
(80.045, 148.01)
(113.1074, 170.79)
(114.1107, 170.79)
(89.5068, 1.54)
(118.98299999999999, 61.52)
(183.9888, 136.77)
(154.9708, 16.04)
(178.9843, 10.2)
(140.9913, 151.33)


# Conclusion

Ground truth in this dataset has 82 (-8) features that are certified by at least one tool.

All viusally inspected to be true. 

Asari table has 3602 features, 71/74 matched to ground truth.

3 missed peaks: two are too below min peak height; one has bad shape, missing 1st half

XCMS table has 4754 features, 60/74 matched to ground truth.
